 # Cours: Applied Data Science Capstone
## Section 0 : 
### The section 0 of this notebook is created for Capstone_Coursera project as just 
### the first test of notebooke creation and check sharing in GiyHub
### Other sections will be following the cours amendes
 

In [181]:
import pandas as pd
import numpy as np

In [182]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


 # Cours: Applied Data Science Capstone
## section 1: 
### Segmenting and Clustering Neighborhoods in Toronto


### section 1-1
#### Transfer data of Toronto's  neighborhoods  from wikipedia to a dataframe

In [183]:
import pandas as pd
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df_TNeib = pd.read_html(url, attrs={"class": "wikitable"})[0] # 0 is for the 1st table in this particular page
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Section 1-2
#### Ignore cells with a borough that is Not assigned

In [184]:
df_TNeib.drop(df_TNeib.index[df_TNeib['Borough'] == 'Not assigned'], inplace = True)
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Section 1-3
#### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11  in the above table.

In [185]:
print ("this section is not applicable as the latest version \n of wikipedia is in the same format as we need.")

this section is not applicable as the latest version 
 of wikipedia is in the same format as we need.


### Section 1-4:
#### 1-4-1 If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.

In [186]:
df_TNeib['Neighbourhood'] = np.where(df_TNeib['Neighbourhood']=='Not assigned', df_TNeib['Borough'], df_TNeib['Neighbourhood'])
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### 1-4-2 Reset index

In [187]:
df_TNeib.reset_index(drop=True, inplace=True)
df_TNeib.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### Section 1-5:
#### Shape of dataframe


In [188]:
df_TNeib.shape

(103, 3)

# Cours: Applied Data Science Capstone
## Section 2
### Section 2-1 
#### Read csv file of postalcodes

In [189]:
df_PCod = pd. read_csv('https://cocl.us/Geospatial_data')
df_PCod.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [190]:
df_PCod.shape

(103, 3)

### Section 2-2 
#### Creat a data frame which connects df_Neib to related Latitude and longitude for every neighbourhood

In [191]:
df_TNeib_LL = pd.merge(df_TNeib,
                 df_PCod[['Postal Code', 'Latitude', 'Longitude']],
                 on='Postal Code')
df_TNeib_LL.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Cours: Applied Data Science Capstone
## Section 3
### Section 3-1 
#### Section 3-1-1
#### first of all , we are importing necessary libraries also reviewing our neighbourhoods on map to have an idea about subject

In [81]:
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install folium
import folium # map rendering library


#### List of bourghs of Toronto

In [192]:
df_Boroughs=df_TNeib_LL.drop_duplicates(subset=['Borough'], keep='first')[['Borough']]
df_Boroughs.reset_index(drop=True, inplace=True)
df_Boroughs

,Borough
0,North York
1,Downtown Toronto
2,Etobicoke
3,Scarborough
4,East York
5,York
6,East Toronto
7,West Toronto
8,Central Toronto
9,Mississauga


#### We have 10 bourgh, for visualisation reason, we need 10 color to apply them on map 
#### We execute color codes generator and we will test on visualiation 
#### If the colors are visually distinguable we make a stable list of these approved colors
#### here is the color codes generator:

In [200]:
import matplotlib.pyplot as plt
import random

number_of_colors = 10

color = ["#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
             for i in range(number_of_colors)]
print(color)

['#12FE25', '#1A1048', '#FE9759', '#C468C2', '#CE3607', '#D5756A', '#96122B', '#5C102E', '#39D808', '#31A3BA']


#### This color list worked well for me so I'll use it for next steps.
##### ['#E6CA69', '#1A395A', '#51FEC6', '#2BB410', '#CD936B', '#A01758', '#8205D8', '#71BA7C', '#00BEF5', '#CE3607']
#### then we assign a color to each of Borough in our data frame by adding a color column , 
#### that means on map we will observe different Borough by different colors

In [213]:
color = ['#E6CA69', '#1A395A', '#51FEC6', '#2BB410', '#CD936B', '#A01758', '#8205D8', '#71BA7C', '#00BEF5', '#CE3607']
df_Boroughs['Color']=df_Boroughs['Borough']
BoroughNames = df_Boroughs["Color"].tolist()
ColorDictionary = {BoroughNames[i]: color[i] for i in range(len(BoroughNames))} 
df_TNeib_LL_Color = df_TNeib_LL
df_TNeib_LL_Color['Color'] = df_TNeib_LL_Color['Borough']
df_TNeib_LL_Color.replace({"Color": ColorDictionary}, inplace=True) 

#### Now we could use the Torono adress to find its coordinate then trace a map which include all neighbourhood circles with different color per bourgh

In [214]:
address = 'Toronto , ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

# create map of New York using latitude and longitude values
map_ = folium.Map(location=[latitude, longitude], zoom_start=9)

# add markers to map
for lat, lng, borough, neighborhood,colorName in zip(df_TNeib_LL_Color['Latitude'], df_TNeib_LL_Color['Longitude'], df_TNeib_LL_Color['Borough'], df_TNeib_LL_Color['Neighbourhood'], df_TNeib_LL_Color['Color']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    #print(colorName)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color= colorName, # 'red', #'blue',
        fill=True,
        fill_color = colorName, #   '#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)  
        #print ([colorName])
map_

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


#### Just to check why we have a isolated circle in left of map, it is is for mississauga so we check and
#### as we observe there is just one line so it is normal that we have just one circle for this bourgh:

In [199]:
df_TNeib_LL_Color[df_TNeib_LL_Color['Borough'] == 'Mississauga']

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Color
76,M7R,Mississauga,Canada Post Gateway Processing Centre,43.636966,-79.615819,#BF3B6F


### Section 3-2 
#### Let explore a some of bourghs like, we are going with thes four :  
#### Downtown Toronto, East Toronto, West Toronto, Central Toronto
### Section 3-2-1 : 
#### Downtown Toronto : what are the neighbourhoods of this bourgh?

In [59]:
df_Downtown_Toronto=df_TNeib_LL[(df_TNeib_LL['Borough'] == 'Downtown Toronto')]
df_Downtown_Toronto.reset_index(drop=True, inplace=True)
print(df_Downtown_Toronto.shape)
df_Downtown_Toronto.head()

(19, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306


In [65]:
address = 'Toronto , ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [71]:
# create map of New York using latitude and longitude values
map_ = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_TNeib_LL['Latitude'], df_TNeib_LL['Longitude'], df_TNeib_LL['Borough'], df_TNeib_LL['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_)  
    
map_

### Section 3-2-2 :
#### East Toronto

In [56]:
df_East_Toronto=df_TNeib_LL[(df_TNeib_LL['Borough'] == 'East Toronto')]
df_East_Toronto.reset_index(drop=True, inplace=True)
print(df_East_Toronto.shape)
df_East_Toronto

(5, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558


### Section 3-2-3 : 
#### West Toronto

In [57]:
df_West_Toronto=df_TNeib_LL[(df_TNeib_LL['Borough'] == 'West Toronto')]
df_West_Toronto.reset_index(drop=True, inplace=True)
print(df_West_Toronto.shape)
df_West_Toronto

(6, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M6H,West Toronto,"Dufferin, Dovercourt Village",43.669005,-79.442259
1,M6J,West Toronto,"Little Portugal, Trinity",43.647927,-79.419750
2,M6K,West Toronto,"Brockton, Parkdale Village, Exhibition Place",43.636847,-79.428191
3,M6P,West Toronto,"High Park, The Junction South",43.661608,-79.464763
4,M6R,West Toronto,"Parkdale, Roncesvalles",43.648960,-79.456325
5,M6S,West Toronto,"Runnymede, Swansea",43.651571,-79.484450


### Section 3-2-4 : 
#### Central Toronto

In [58]:
df_Central_Toronto=df_TNeib_LL[(df_TNeib_LL['Borough'] == 'Central Toronto')]
df_Central_Toronto.reset_index(drop=True, inplace=True)
print(df_Central_Toronto.shape)
df_Central_Toronto

(9, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
1,M5N,Central Toronto,Roselawn,43.711695,-79.416936
2,M4P,Central Toronto,Davisville North,43.712751,-79.390197
3,M5P,Central Toronto,"Forest Hill North & West, Forest Hill Road Park",43.696948,-79.411307
4,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
5,M5R,Central Toronto,"The Annex, North Midtown, Yorkville",43.672710,-79.405678
6,M4S,Central Toronto,Davisville,43.704324,-79.388790
7,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
8,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049
